2번 실험 

In [ ]:
import streamlit as st
import random
import datetime
import time

# Set up the page configuration
st.set_page_config(
    page_title="Main Page",
    page_icon="👋",
)

# Set up a title for the main page
st.title("Welcome to the Guessing Game App")

# Check if the 'game_user' is set in session state
if 'game_user' not in st.session_state:
    st.session_state.game_user = None

if st.session_state.game_user is None:
    # If the user is not set, ask for their name
    st.title("Welcome to the Guessing Game!")
    st.write("Please enter your name to start playing.")

    # Input field for the user's name
    user_name = st.text_input("Enter your name", key="uuuuuuu")

    # Button to confirm the name
    if st.button("Start Game"):
        if user_name:
            # Save the name to session state
            st.session_state.game_user = user_name
            st.success(f"Welcome, {st.session_state.game_user}! Let's start playing!")
        else:
            st.error("Please enter a right name to proceed.")
else:
    # Now that the user has entered their name, show the available pages
    st.title(f"Hello, {st.session_state.game_user}!")
    st.write("You can now access the game pages using the navigation sidebar.")

home_page = st.Page("app3.py", title="Home", icon="🏠")
game_page = st.Page("pages/2_play.py", title="Play the guessing game", icon="🕹️")
stats_page = st.Page("pages/3_stats.py", title="See the stats", icon="📊")

# Create a navigation bar
pg = st.navigation([home_page,game_page, stats_page])

# Run the app
pg.run()

APP3 실험

In [ ]:
import streamlit as st
import random
from dotenv import load_dotenv
import os
from openai import OpenAI


# Load environment variables from the .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)


if "logged_in" not in st.session_state:
    st.session_state.logged_in = False

def login():
    if st.button("Log in"):
        st.session_state.logged_in = True
        st.rerun()

def logout():
    if st.button("Log out"):
        st.session_state.logged_in = False
        st.rerun()

login_page = st.Page(login, title="Log in", icon=":material/login:")
logout_page = st.Page(logout, title="Log out", icon=":material/logout:")

game_page = st.Page("pages/2_play.py", title="Play the guessing game", icon="🕹️")
stats_page = st.Page("pages/3_stats.py", title="See the stats", icon="📊")


if st.session_state.logged_in:
    pg = st.navigation(
        {
            "Account": [logout_page],
            "Games": [game_page, stats_page],
        }
    )
else:
    pg = st.navigation([login_page])

pg.run()

모든 페이지 하나로 넣는 어프로치 

In [ ]:
import streamlit as st
import random
from dotenv import load_dotenv
import os
from openai import OpenAI


# Load environment variables from the .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

# Set up a title for the main page
st.title("Welcome to the Guessing Game App")

# Check if the 'game_user' is set in session state
if 'game_user' not in st.session_state:
    st.session_state.game_user = None

# Sidebar navigation
page = st.sidebar.selectbox("Select a page", ["Home", "Play", "Stats"])


# Handle the Home page and user name input
if page == "Home":
    if st.session_state.game_user is None:
        st.write("Please enter your name to start playing.")

        # Input field for the user's name
        user_name = st.text_input("Enter your name", key="nnnnnn")

        # Button to confirm the name
        if st.button("Start Game"):
            if user_name:
                st.session_state.game_user = user_name
                st.success(f"Welcome, {st.session_state.game_user}! Let's start playing!")
            else:
                st.error("Please enter a valid name to proceed.")
    else:
        # User has entered their name
        st.write(f"Hello, {st.session_state.game_user}!")
        st.write("You can now access the game pages using the navigation sidebar.")

# Handle the Play page
elif page == "Play":
    st.title(f"Hi! {st.session_state.game_user}! Here we are playing the animal guessing game!")
    st.write("I am thinking of an animal, and you will ask me questions to guess it. "
             "I only answer with 'yes' or 'no'.")

    if "game_started" not in st.session_state:
        st.session_state.game_started = True
        st.session_state.chat_history = []
        st.session_state.animal = random.choice(
            ["cat", "dog", "elephant", "lion", "tiger"])  # Random animal for this game

    st.title("Animal Guessing Game")
    st.write("Guess the animal I'm thinking of!")

    # Displaying chat history
    for msg in st.session_state.chat_history:
        st.chat_message(msg["role"]).markdown(msg["message"])

    # User input for guessing the animal
    user_guess = st.text_input("What is your guess? (Type the animal name)", key="user_guess")

    if user_guess:
        # AI responds with 'yes' or 'no'
        if user_guess.lower() == st.session_state.animal.lower():
            ai_response = "Yes, that's correct! Well done!"
        else:
            ai_response = "No, that's not correct. Try again!"

        # Append the user guess and AI response to the chat history
        st.session_state.chat_history.append({"role": "user", "message": f"Is it a {user_guess}?"})
        st.session_state.chat_history.append({"role": "assistant", "message": ai_response})

        # Display AI response
        st.chat_message("assistant").markdown(ai_response)

        # Option to start a new game
        if ai_response.startswith("Yes"):
            if st.button("Start a New Game"):
                st.session_state.game_started = False
                st.session_state.chat_history = []



# Handle the Stats page
elif page == "Stats":
    st.title("Game Stats")
    # Example stats (you can track and display more meaningful data here)
    st.write("Total games played: 1")
    st.write("Total guesses so far: ", st.session_state.get('guesses', 0))
    st.write("Average guesses per game: ", round(random.uniform(1, 5), 2))  # Example random data
    st.bar_chart([random.randint(1, 10) for _ in range(5)])  # Example bar chart data
